In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [21]:
image_height, image_width = 64, 64

In [22]:
def load_data(folder_path):
    X = []
    y = []
    for label, subfolder in enumerate(['normal', 'abnormal']):
        subfolder_path = os.path.join(folder_path, subfolder)
        for image_filename in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, image_filename)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (image_height, image_width))
            image = image.astype('float32') / 255.0  # Normalize pixel values
            X.append(image)
            y.append(label)
    return np.array(X), np.array(y)

dataset_path = 'dataset'
X, y = load_data(dataset_path)
X
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])

In [23]:
# Step 3: Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [24]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [25]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [26]:
# Step 6: Train Model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 368ms/step - accuracy: 0.3615 - loss: 0.7074 - val_accuracy: 0.3846 - val_loss: 0.7072
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.5058 - loss: 0.6394 - val_accuracy: 0.5385 - val_loss: 0.6801
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.7295 - loss: 0.5832 - val_accuracy: 0.8462 - val_loss: 0.5352
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.8882 - loss: 0.4738 - val_accuracy: 0.7436 - val_loss: 0.5695
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.9441 - loss: 0.3450 - val_accuracy: 0.8974 - val_loss: 0.2828
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.8660 - loss: 0.2829 - val_accuracy: 0.7179 - val_loss: 0.6846
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.9558 - loss: 0.1849 - val_accuracy: 0.9231 - val_loss: 0.1794
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.9115 - loss: 0.2059 - val_accuracy: 0.6410 - val_loss:

In [27]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', test_accuracy)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9450 - loss: 0.1007
Test accuracy: 0.9487179517745972


In [28]:
import cv2
import numpy as np

sample_image = 'abnormal10_9900.png'

# Load the image
image = cv2.imread(sample_image)

# Check if the image is loaded successfully
if image is None:
    print(f"Failed to load {sample_image}")
else:
    # Resize the image
    image = cv2.resize(image, (image_height, image_width))

    # Check if the image is resized successfully
    if image is None:
        print(f"Failed to resize {sample_image}")
    else:
        # Normalize pixel values
        image = image.astype('float32') / 255.0  

        # Add batch dimension
        image = np.expand_dims(image, axis=0)  

        # Make prediction
        prediction = model.predict(image)

        # Print prediction
        if prediction[0][0] > 0.5:
            print(sample_image, 'is ABNORMAL')
        else:
            print(sample_image, 'is NORMAL')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
abnormal10_9900.png is ABNORMAL
